In [63]:
import requests
from datetime import datetime, UTC
from langchain.agents import tool
from pydantic import BaseModel, Field

latitude = -23.6865
longitude = -46.6234


class RetornaTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')

@tool(args_schema=RetornaTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para cada coordenada'''
    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly" : "temperature_2m",
        'forecast_days': 1
    }
    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()

        hora_agora = datetime.now(UTC).replace(tzinfo=None)
        lista_horas = [datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x]-hora_agora))
        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]

        return temp_atual
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')
    

In [65]:
retorna_temperatura_atual.args

{'latitude': {'description': 'Latitude da localidade que buscamos a temperatura',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude da localidade que buscamos a temperatura',
  'title': 'Longitude',
  'type': 'number'}}

In [66]:
retorna_temperatura_atual.invoke({'latitude' : latitude, 'longitude': longitude})

20.5

In [84]:
import wikipedia
wikipedia.set_lang('pt')

query = 'Isaac Asimov'

class search_wiki(BaseModel):
    query : str = Field(description='Pergunta ou assunto que deve ser localizado no wikipedia')

@tool(args_schema=search_wiki)
def busca_wikipedia(query:str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query feita"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

busca_wikipedia.invoke({'query':'Isaac ASIMOV'})

'Título da página: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus livro

## Testando a Tool

In [99]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente amigável chamado Joselito, expert em conhecimentos gerais.'),
        ('user','{input}')
    ]
)

chat = ChatOpenAI()

tools = [busca_wikipedia, retorna_temperatura_atual]
tool_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}


# chain = prompt | chat.bind(functions=tools)

In [94]:
tool_run['busca_wikipedia'].invoke({'query':'Isaac Asimov'})

'Título da página: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus livro

# Criando Routers

In [102]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tool_json) | OpenAIFunctionsAgentOutputParser()

In [103]:
chain.invoke({'input': 'Olá'})

AgentFinish(return_values={'output': 'Olá! Como posso te ajudar hoje?'}, log='Olá! Como posso te ajudar hoje?')

# Rodando as ferramentas

Podemos criar uma função simples de roteamento para lidar com os dois estados possíveis

In [104]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [105]:
chain = prompt | chat.bind(functions = tool_json ) | OpenAIFunctionsAgentOutputParser() | roteamento

ARviv

In [117]:
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class ArvixArgs(BaseModel):
    query: str = Field(description='Query de busca no Arxiv')

tool_arxiv = StructuredTool.from_function(
    func=ArxivAPIWrapper(top_k_results=2).run,
    name='arxiv',
    description = (
        "Uma ferramenta em torno do Arxiv.org."
        "Útil para quando você precisa responder a pergunta sobre Física, Matemática, "
        "Ciência da computação, Biologia Quantitativa, Finanças Quantitativas, Estatística, "
        "utilizando artigos científicos do arxiv.org. "
        "A entrada deve ser uma consulta de pesquisa em inglês."
    ),
    return_direct=True
)

In [110]:
print({'Descrição': tool_arxiv.description})
print({'Args': tool_arxiv.args})

{'Descrição': 'Uma ferramenta em torno do Arxiv.org.Útil para quando você precisa responder a pergunta sobre Física, Matemática, Ciência da computação, Biologia Quantitativa, Finanças Quantitativas, Estatística, utilizando artigos científicos do arxiv.org. A entrada deve ser uma consulta de pesquisa em inglês.'}
{'Args': {'arxiv_search': {'title': 'Arxiv Search'}, 'arxiv_exceptions': {'title': 'Arxiv Exceptions'}, 'top_k_results': {'default': 3, 'title': 'Top K Results', 'type': 'integer'}, 'ARXIV_MAX_QUERY_LENGTH': {'default': 300, 'title': 'Arxiv Max Query Length', 'type': 'integer'}, 'continue_on_failure': {'default': False, 'title': 'Continue On Failure', 'type': 'boolean'}, 'load_max_docs': {'default': 100, 'title': 'Load Max Docs', 'type': 'integer'}, 'load_all_available_meta': {'default': False, 'title': 'Load All Available Meta', 'type': 'boolean'}, 'doc_content_chars_max': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': 4000, 'title': 'Doc Content Chars Max'}}}


In [118]:
tool_arxiv.run({'query':'O que é LLM?'})

"Published: 2024-11-02\nTitle: Interacting Large Language Model Agents. Interpretable Models and Social Learning\nAuthors: Adit Jain, Vikram Krishnamurthy\nSummary: This paper develops theory and algorithms for interacting large language\nmodel agents (LLMAs) using methods from statistical signal processing and\nmicroeconomics. While both fields are mature, their application to\ndecision-making by interacting LLMAs remains unexplored. Motivated by Bayesian\nsentiment analysis on online platforms, we construct interpretable models and\nstochastic control algorithms that enable LLMAs to interact and perform\nBayesian inference. Because interacting LLMAs learn from prior decisions and\nexternal inputs, they exhibit bias and herding behavior. Thus, developing\ninterpretable models and stochastic control algorithms is essential to\nunderstand and mitigate these behaviors. This paper has three main results.\nFirst, we show using Bayesian revealed preferences from microeconomics that an\nindi

Instanciando tool já criada no LangChain

In [119]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=3))

Criando atraves do load_tools

In [120]:
from langchain.agents import load_tools

In [122]:
tools = load_tools(['arxiv'])
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))]

Python REPL

In [125]:
from langchain_experimental.tools.python.tool import PythonAstREPLTool

tool_repl = PythonAstREPLTool()
print({'Description': tool_repl.description})
print({'Args': tool_repl.args})

{'Description': 'A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.'}
{'Args': {'query': {'description': 'code snippet to run', 'title': 'Query', 'type': 'string'}}}


In [127]:
tool_repl.run({'query':'print("oi")'})

'oi\n'

StackOverflow

In [131]:
from langchain.agents import load_tools

tools = load_tools (['stackexchange'])
tool_stack = tools[0]
print('Descrição:', tool_stack.description)
print('Args', tool_stack.args)

Descrição: A wrapper around StackExchange. Useful for when you need to answer specific programming questionscode excerpts, code examples and solutionsInput should be a fully formed question.
Args {'query': {'title': 'Query', 'type': 'string'}}


In [134]:
tool_stack.run({'query':'langchain agents'})

'Question: SQLalchemy and Langchain: How to select a specific table?\nI am new to data engineering and I&#39;m trying to use the sql <span class="highlight">agent</span> in <span class="highlight">Langchain</span> but I cannot seem to ingest the sql table correctly. &hellip; \n\nQuestion: Modify system prompt for ReAct agent (langchain)\nI have a React <span class="highlight">agent</span> and I use a prompt from the langsmith hub. &hellip; This is my code\n# ReAct <span class="highlight">agent</span> prompt\n    prompt = hub.pull(&quot;hwchase17/structured-chat-<span class="highlight">agent</span>&quot;)\n\n    # ConversationBufferMemory stores the conversation history in RAM\n    memory = ConversationBufferMemory &hellip; \n\nQuestion: Integrating llama index vectorstoreindex with Langchain agents for RAG Applications\nI want to use <span class="highlight">LangChain</span> because of its functionality for developing more complex prompt templates (similarly I have not really investigat

File System

In [136]:
from langchain_community.agent_toolkits.file_management.toolkit import FileManagementToolkit

tool_kit = FileManagementToolkit(
    root_dir='arquivos'
)
tools = tool_kit.get_tools()
for tool in tools:
    print('\nname :', tool.name)
    print('description : ', tool.description)
    print('args : ', tool.args)


name : copy_file
description :  Create a copy of a file in a specified location
args :  {'source_path': {'description': 'Path of the file to copy', 'title': 'Source Path', 'type': 'string'}, 'destination_path': {'description': 'Path to save the copied file', 'title': 'Destination Path', 'type': 'string'}}

name : file_delete
description :  Delete a file
args :  {'file_path': {'description': 'Path of the file to delete', 'title': 'File Path', 'type': 'string'}}

name : file_search
description :  Recursively search for files in a subdirectory that match the regex pattern
args :  {'dir_path': {'default': '.', 'description': 'Subdirectory to search in.', 'title': 'Dir Path', 'type': 'string'}, 'pattern': {'description': 'Unix shell regex, where * matches everything.', 'title': 'Pattern', 'type': 'string'}}

name : move_file
description :  Move or rename a file from one location to another
args :  {'source_path': {'description': 'Path of the file to move', 'title': 'Source Path', 'type': '

## Dando um exemplo de aplicação

In [146]:
from langchain_community.agent_toolkits.file_management.toolkit import FileManagementToolkit

tool_kit = FileManagementToolkit(
    root_dir='arquivos',
    selected_tools = ['write_file','read_file','file_search','list_directory']
)
tools = tool_kit.get_tools()

In [147]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

tools_json=[convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name : tool for tool in tools}

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Hemoraldo capaz de gerenciar arquivos'),
    ('user', '{input}')
])

chat = ChatOpenAI()

In [148]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [153]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | roteamento

In [154]:
chain.invoke({'input': 'O que você pode fazer?'})

'Posso ajudar a gerenciar arquivos, como escrever em arquivos, ler arquivos, pesquisar arquivos por padrões de regex e listar arquivos e diretórios em uma pasta específica. Se precisar de alguma dessas funções, é só me dizer como posso te ajudar!'

In [155]:
chain.invoke({'input': 'Quais arquivos em pdf tem na pasta?'})

'Explorando o Universo das IAs com Hugging Face.pdf'

In [157]:
chain.invoke({'input': 'Crie um arquivo txt  e salve neste diretório e diga que foi feito por uma LLM'})

'File written successfully to LLM.txt.'